 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

This is simply a notebook to help me test stuff for the preprocessing script.

In [1]:
import netCDF4 as nc
import os

In [67]:
from subprocess import getoutput

In [68]:
s = getoutput(['pwd']).rstrip()
s

'/Users/stephanrasp/repositories/CBRAIN/data_processing'

In [66]:
s.split('/')

["b'", 'Users', 'stephanrasp', 'repositories', 'CBRAIN', "data_processing'"]

In [2]:
data_dir = '/Volumes/STICK/data/SPCAM/Aquaplanet/'
%ls $data_dir

AndKua_aqua_SPCAM3.0.cam2.h1.0000-01-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-02-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-03-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-04-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-06-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-09-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-12-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0001-01-01-00000.nc*


In [3]:
rg = nc.Dataset(data_dir + 'AndKua_aqua_SPCAM3.0.cam2.h1.0000-01-01-00000.nc')
rg

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    source: CAM
    case: AndKua_aqua_SPCAM3.0
    title: 
    logname: tg802402
    host: c532-003.stamped
    Version: $Name:  $
    revision_Id: $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $
    dimensions(sizes): lat(64), lon(128), lev(30), ilev(31), isccp_prs(7), isccp_tau(7), isccp_prstau(49), time(48), tbnd(2), chars(8), crm_x(32), crm_y(1), crm_z(28)
    variables(dimensions): float64 P0(), float64 lat(lat), float64 lon(lon), float64 crm_x(crm_x), float64 crm_y(crm_y), float64 crm_z(crm_z), float64 lev(lev), float64 ilev(ilev), float64 isccp_prs(isccp_prs), float64 isccp_tau(isccp_ta), float64 isccp_prstau(isccp_prsta), float64 time(time), float64 time_bnds(time,tbnd), |S1 date_written(time,chars), |S1 time_written(time,chars), int32 ntrm(), int32 ntrn(), int32 ntrk(), int32 ndbase(), int32 nsbase(), int32 nbdate(), int32 nbsec(), int32 mdt(),

In [57]:
rg.variables['PHQ']

<class 'netCDF4._netCDF4.Variable'>
float32 PHQ(time, lev, lat, lon)
    units: kg/kg/s
    long_name: Specific humidity physics tendency
    cell_method: time: mean
unlimited dimensions: time
current shape = (48, 30, 64, 128)
filling off

In [55]:
time = rg.variables['TAP']
time.shape

(48, 30, 64, 128)

In [5]:
time[:]

array([ 0.        ,  0.02083333,  0.04166667,  0.0625    ,  0.08333333,
        0.10416667,  0.125     ,  0.14583333,  0.16666667,  0.1875    ,
        0.20833333,  0.22916667,  0.25      ,  0.27083333,  0.29166667,
        0.3125    ,  0.33333333,  0.35416667,  0.375     ,  0.39583333,
        0.41666667,  0.4375    ,  0.45833333,  0.47916667,  0.5       ,
        0.52083333,  0.54166667,  0.5625    ,  0.58333333,  0.60416667,
        0.625     ,  0.64583333,  0.66666667,  0.6875    ,  0.70833333,
        0.72916667,  0.75      ,  0.77083333,  0.79166667,  0.8125    ,
        0.83333333,  0.85416667,  0.875     ,  0.89583333,  0.91666667,
        0.9375    ,  0.95833333,  0.97916667])

In [6]:
lats = rg.variables['lat'][:]

In [34]:
lats.shape

(64,)

In [46]:
e = lats.reshape((1, 64, 1))
e.shape

(1, 64, 1)

In [47]:
e = np.repeat(e, 5, axis=0)

In [48]:
e.shape

(5, 64, 1)

In [49]:
e = np.repeat(e, 10, axis=2)
e.shape

(5, 64, 10)

In [52]:
e[0, 0, :]

array([-87.86379884, -87.86379884, -87.86379884, -87.86379884,
       -87.86379884, -87.86379884, -87.86379884, -87.86379884,
       -87.86379884, -87.86379884])

In [37]:
c = np.repeat(lats, 5 * 10)
c.shape

(3200,)

In [43]:
d = c.reshape((5, 64, 10))

In [44]:
d.shape

(5, 64, 10)

In [45]:
d[0,:,0]

array([-87.86379884, -87.86379884, -87.86379884, -87.86379884,
       -87.86379884, -85.09652699, -85.09652699, -85.09652699,
       -85.09652699, -85.09652699, -82.31291295, -82.31291295,
       -82.31291295, -82.31291295, -82.31291295, -79.52560657,
       -79.52560657, -79.52560657, -79.52560657, -79.52560657,
       -76.73689968, -76.73689968, -76.73689968, -76.73689968,
       -76.73689968, -73.94751515, -73.94751515, -73.94751515,
       -73.94751515, -73.94751515, -71.15775201, -71.15775201,
       -71.15775201, -71.15775201, -71.15775201, -68.36775611,
       -68.36775611, -68.36775611, -68.36775611, -68.36775611,
       -65.57760701, -65.57760701, -65.57760701, -65.57760701,
       -65.57760701, -62.7873518 , -62.7873518 , -62.7873518 ,
       -62.7873518 , -62.7873518 , -59.99702011, -59.99702011,
       -59.99702011, -59.99702011, -59.99702011, -57.20663153,
       -57.20663153, -57.20663153, -57.20663153, -57.20663153,
       -54.41619953, -54.41619953, -54.41619953, -54.41

In [7]:
cond = (lats > -90) & (lats < 90)
cond

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [8]:
import numpy as np

In [9]:
lat_idxs = np.where((lats > -90) & (lats < 90))

In [10]:
lat_idxs

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),)

In [11]:
tap = rg.variables['TAP']

In [29]:
'LAT' in rg.variables.keys()

False

In [12]:
tap.dimensions

('time', 'lev', 'lat', 'lon')

In [15]:
tap[:][:, :, lat_idxs, :]

array([[[[[ 246.80204773,  246.79644775,  246.79115295, ...,  246.81835938,
            246.81265259,  246.80718994],
          [ 246.8709259 ,  246.86141968,  246.85159302, ...,  246.89523315,
            246.88807678,  246.87986755],
          [ 246.9909668 ,  246.96942139,  246.94703674, ...,  247.04100037,
            247.02748108,  247.01074219],
          ..., 
          [ 167.77011108,  167.75193787,  167.73597717, ...,  167.84043884,
            167.81425476,  167.79090881],
          [ 167.94969177,  167.94126892,  167.93696594, ...,  167.99821472,
            167.97850037,  167.96220398],
          [ 168.68197632,  168.68193054,  168.68389893, ...,  168.69444275,
            168.6882782 ,  168.68406677]]],


        [[[ 233.19773865,  233.19470215,  233.19039917, ...,  233.19779968,
            233.19889832,  233.1988678 ],
          [ 233.290802  ,  233.29478455,  233.29711914, ...,  233.27140808,
            233.27894592,  233.28540039],
          [ 233.34965515,  233.37680

In [16]:
a = np.arange(20)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [24]:
b = np.arange(0, 0+19)
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [25]:
a[b]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [9]:
nc.Dataset(data_dir + 'AndKua_aqua_SPCAM3.0.cam2.h1.0001-01-01-00000.nc').variables['time'][:]

array([ 365.        ,  365.02083333,  365.04166667,  365.0625    ,
        365.08333333,  365.10416667,  365.125     ,  365.14583333,
        365.16666667,  365.1875    ,  365.20833333,  365.22916667,
        365.25      ,  365.27083333,  365.29166667,  365.3125    ,
        365.33333333,  365.35416667,  365.375     ,  365.39583333,
        365.41666667,  365.4375    ,  365.45833333,  365.47916667,
        365.5       ,  365.52083333,  365.54166667,  365.5625    ,
        365.58333333,  365.60416667,  365.625     ,  365.64583333,
        365.66666667,  365.6875    ,  365.70833333,  365.72916667,
        365.75      ,  365.77083333,  365.79166667,  365.8125    ,
        365.83333333,  365.85416667,  365.875     ,  365.89583333,
        365.91666667,  365.9375    ,  365.95833333,  365.97916667])

In [10]:
nc.Dataset('/Volumes/STICK/data/SPCAM/sample/SPCAM_outputs.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): t(100000), z(21)
    variables(dimensions): float64 LAT(t), float64 PS(t), float64 SHFLX(t), float64 LHFLX(t), float64 OMEGA(z,t), float64 QAP(z,t), float64 TAP(z,t), float64 QRL(z,t), float64 QRS(z,t), float64 UBSP(z,t), float64 VBSP(z,t), float64 SPQC(z,t), float64 SPQG(z,t), float64 SPQI(z,t), float64 SPQR(z,t), float64 SPMC(z,t), float64 SPMCDN(z,t), float64 SPMCUDN(z,t), float64 SPMCUP(z,t), float64 SPMCUUP(z,t), float64 TPHYSTND(z,t), float64 PHQ(z,t), float64 SPDT(z,t), float64 SPDQ(z,t), float64 SPDQC(z,t), float64 SPDQI(z,t), float64 dTdt_adiabatic(z,t), float64 dQdt_adiabatic(z,t), float64 GRAD_UQ_H(z,t)
    groups: 

In [13]:
64 * 128 * 48

393216

In [16]:
4.5e6 / (64 * 128 * 48)

11.444091796875